In [1]:
!pip install gradio rembg onnxruntime pytesseract opencv-python --quiet

import gradio as gr
from rembg import remove
from PIL import Image
import pytesseract, re, cv2
import numpy as np

def format_date(ymd):
    return f"{ymd[4:6]}-{ymd[2:4]}-{ymd[0:2]}"

def parse_mrz(text):
    lines = [line.strip().replace(' ', '').replace('K', '<') for line in text.splitlines() if line.strip()]
    def is_mrz_line(line): return re.fullmatch(r'[A-Z0-9<]{20,}', line) is not None
    for i in range(len(lines) - 2):
        l1, l2, l3 = lines[i:i+3]
        if all(is_mrz_line(l) for l in [l1, l2, l3]):
            id_match = re.search(r'[A-Z]{3}([0-9]{8})<', l1)
            id_number = id_match.group(1) if id_match else None
            dob_raw = l2[0:6]
            gender = l2[7]
            expiry_raw = l2[8:14]
            try:
                dob = format_date(dob_raw)
                expiry = format_date(expiry_raw)
            except:
                dob = expiry = "Invalid"
            name_parts = l3.split('<', 1)
            first = name_parts[0].replace('<', '')
            last = name_parts[1].replace('<', '') if len(name_parts) > 1 else ''
            return {
                "ID Number": id_number,
                "Date of Birth": dob,
                "Gender": gender,
                "Expiry Date": expiry,
                "First Name": first[:25],
                "Last Name": last[:25]
            }
    return {"error": "MRZ block not found"}

def extract_mrz(image):
    no_bg = remove(image)
    np_img = np.array(no_bg)

    if np_img.shape[2] == 4:
        alpha = np_img[:, :, 3]
        non_empty_cols = np.where(np.max(alpha, axis=0) > 0)[0]
        non_empty_rows = np.where(np.max(alpha, axis=1) > 0)[0]
        if non_empty_cols.size and non_empty_rows.size:
            crop_box = (
                non_empty_cols[0], non_empty_rows[0],
                non_empty_cols[-1], non_empty_rows[-1]
            )
            cropped_image = no_bg.crop(crop_box)
            cropped_image = cropped_image.resize((300, int(cropped_image.height * 300 / cropped_image.width)))
        else:
            cropped_image = no_bg
    else:
        cropped_image = no_bg

    image_np = np.array(cropped_image.convert("RGB"))
    image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789<'
    text = pytesseract.image_to_string(gray, config=config)
    text = re.sub(r'(?<=[A-Z0-9])K(?=[A-Z0-9<])', '<', text)
    result = parse_mrz(text)

    return cropped_image, result

# 🔗 This gives you a PUBLIC URL
gr.Interface(
    fn=extract_mrz,
    inputs=gr.Image(type="pil", label="Upload your ID/passport"),
    outputs=["image", "json"],
    title="MRZ Extractor",
    description="Extracts MRZ and parses passport/ID data.",
).launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cde2ea0693135099cb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
